In [2]:
import sys
import yaml
import itertools
import xarray as xr
import numpy as np
import pandas as pd
%matplotlib inline
from functools import partial
# import wrf
import metpy.calc as mpcalc
from metpy.units import units


# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from utils import select_months_ds, generate_ptlst_from_start_end
from statistical_tests import ttest_1samp_new

In [3]:
path_to_data = '/expanse/nfs/cw3e/cwp140/'
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [4]:
region = 'gulf_of_mexico2' ## 'san_juan' 'baja' 'gulf_of_mexico'
path_to_data = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5/cross_section/'
# fname_pattern = path_to_data + 'anomalies/daily_filtered_anomalies_{0}_*.nc'.format(region)
fname_pattern = path_to_data + 'daily/era5_{0}_025dg_daily_uvwq_*.nc'.format(region)
ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='by_coords')
ds

<xarray.Dataset>
Dimensions:    (location: 13, level: 23, time: 8706)
Coordinates:
    longitude  (location) float32 dask.array<chunksize=(13,), meta=np.ndarray>
    latitude   (location) float32 dask.array<chunksize=(13,), meta=np.ndarray>
  * level      (level) int32 1000 975 950 925 900 875 ... 350 300 250 225 200
  * time       (time) datetime64[ns] 2000-03-01 2000-03-02 ... 2023-12-31
Dimensions without coordinates: location
Data variables:
    q          (time, level, location) float32 dask.array<chunksize=(31, 23, 13), meta=np.ndarray>
    u          (time, level, location) float32 dask.array<chunksize=(31, 23, 13), meta=np.ndarray>
    v          (time, level, location) float32 dask.array<chunksize=(31, 23, 13), meta=np.ndarray>
    w          (time, level, location) float32 dask.array<chunksize=(31, 23, 13), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-11-08 23:15:54 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
# import configuration file for case study choice
yaml_doc = '../data/domains.yml'
d = yaml.load(open(yaml_doc), Loader=yaml.SafeLoader)

coord_pairs = generate_ptlst_from_start_end(d[region]['start_pt'][1], d[region]['start_pt'][0], d[region]['end_pt'][1], d[region]['end_pt'][0], pairs=False)
print(coord_pairs)

(array([26.  , 26.25, 26.5 , 26.75, 27.  , 27.25, 27.5 , 27.75, 28.  ,
       28.25, 28.5 , 28.75, 29.  ]), array([-99. , -98.5, -98. , -97.5, -97. , -96.5, -96. , -95.5, -95. ,
       -94.5, -94. , -93.5, -93. ]))


In [7]:
## load ar dates within region
fname = '../out/line_dates_{0}.csv'.format(region)
df = pd.read_csv(fname)
df['day'] = pd.to_datetime(df['datetime']).dt.normalize()
df = df.sort_values(by=['datetime'])
# df = df.set_index(pd.to_datetime(df['datetime'])) ## set daily values as index
ar_dates = df['day'].values
## create year month columns in dataframe 
# df['yearmonth'] = df.index.strftime("%Y%m")
new = df.drop_duplicates('day')
ar_dates = new['day'].values
new

,Unnamed: 0,datetime,day
24,24,2000-04-28 04:00:00,2000-04-28
63,65,2000-10-22 05:00:00,2000-10-22
33,33,2000-10-27 08:00:00,2000-10-27
6,6,2002-06-19 00:00:00,2002-06-19
30,30,2002-07-03 08:00:00,2002-07-03
...,...,...,...
99,106,2022-07-04 05:00:00,2022-07-04
44,45,2023-05-08 17:00:00,2023-05-08
53,55,2023-05-09 01:00:00,2023-05-09
59,61,2023-07-06 10:00:00,2023-07-06


In [9]:
import pandas as pd

def find_missing_dates(list1, list2):
    """Finds dates in list1 that are not in list2."""

    # Convert lists to datetime objects
    dates1 = pd.to_datetime(list1)
    dates2 = pd.to_datetime(list2)

    # Create a set of all dates in list1
    set1 = set(dates1)

    # Find dates in set1 that are not in list2
    missing_dates = set1 - set(dates2)

    # Convert back to list of strings
    return [date.strftime('%Y-%m-%d') for date in missing_dates]

# Example usage
list1 = ar_dates
list2 = ds.time.values

missing_dates = find_missing_dates(list1, list2)
print(missing_dates)  # Output: ['2023-01-05', '2023-01-02']

['2000-02-23', '2000-01-17']


In [11]:
ds.time.values[:100]

array(['2000-03-01T00:00:00.000000000', '2000-03-02T00:00:00.000000000',
       '2000-03-03T00:00:00.000000000', '2000-03-04T00:00:00.000000000',
       '2000-03-05T00:00:00.000000000', '2000-03-06T00:00:00.000000000',
       '2000-03-07T00:00:00.000000000', '2000-03-08T00:00:00.000000000',
       '2000-03-09T00:00:00.000000000', '2000-03-10T00:00:00.000000000',
       '2000-03-11T00:00:00.000000000', '2000-03-12T00:00:00.000000000',
       '2000-03-13T00:00:00.000000000', '2000-03-14T00:00:00.000000000',
       '2000-03-15T00:00:00.000000000', '2000-03-16T00:00:00.000000000',
       '2000-03-17T00:00:00.000000000', '2000-03-18T00:00:00.000000000',
       '2000-03-19T00:00:00.000000000', '2000-03-20T00:00:00.000000000',
       '2000-03-21T00:00:00.000000000', '2000-03-22T00:00:00.000000000',
       '2000-03-23T00:00:00.000000000', '2000-03-24T00:00:00.000000000',
       '2000-03-25T00:00:00.000000000', '2000-03-26T00:00:00.000000000',
       '2000-03-27T00:00:00.000000000', '2000-03-28

In [6]:
def get_startmon_and_endmon(ssn):
    ## set start_mon and end_mon based on ssn
    if ssn == 'DJF':
        start_mon, end_mon = (12, 2)
    elif ssn == 'MAM':
        start_mon, end_mon = (3, 5)
    elif ssn == 'JJA':
        start_mon, end_mon = (6, 8)
    elif ssn == 'SON':
        start_mon, end_mon = (9, 11)
    elif ssn == 'NDJFMA':
        start_mon, end_mon = (11, 4)
    elif ssn == 'MJJASO':
        start_mon, end_mon = (5, 10)

    return start_mon, end_mon

In [7]:

def compute_freezing_level_composites(varname, ar_dates, ssn, region):
   ## load data
    path_to_data = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5/cross_section/sfc_prs_deg0l/'
    out_path = '/home/dnash/DATA/preprocessed/ERA5_composites/cross_section/'
    
    fname_pattern = path_to_data + "era5_{0}_025dg_hourly_sp_deg0l_*.nc".format(region)
    ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='by_coords')
    
    ## subset to AR dates
    ds = ds.sel(time=ar_dates)
    
    ## subset to start_month and end_month based on ssn
    start_mon, end_mon = get_startmon_and_endmon(ssn)
    tmp = select_months_ds(ds, start_mon, end_mon, 'time')
    tmp = tmp.load()
    
    ## Calculate the percentiles
    quantile_arr = np.array([0.1, .5, .9])
    quantile = tmp.quantile(quantile_arr, dim=['time'], skipna=True)
    
    ## save as netcdf
    out_path = '/home/dnash/DATA/preprocessed/ERA5_composites/cross_section/'
    out_fname = out_path + 'composite_{0}_{1}_{2}.nc'.format(varname, ssn, region)
    quantile.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    return quantile
    
def compute_vertical_composites(varname, anomaly, ar_dates, ssn, region):
    ## function that computes composites (anomaly or non-anomaly) for defined start_mon and end_mon    
    ## for DJF, MAM, JJA, SON, NDJFMA, and MJJASON
    ## compute anomaly composites - anomaly = True
    ## compute non-anomaly composites - anomaly=False

    ## load data
    path_to_data = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5/cross_section/'
    out_path = '/home/dnash/DATA/preprocessed/ERA5_composites/cross_section/'
    
    if anomaly == True:
        fname_pattern = path_to_data + 'anomalies/daily_filtered_anomalies_{0}_*.nc'.format(region)
    else:
        fname_pattern = path_to_data + 'daily/era5_{0}_025dg_daily_uvwq_*.nc'.format(region)

    ds = xr.open_mfdataset(fname_pattern, engine='netcdf4', combine='by_coords')

    ## subset to AR dates
    ds = ds.sel(time=ar_dates)

    ## subset to start_month and end_month
    start_mon, end_mon = get_startmon_and_endmon(ssn)
    ds = select_months_ds(ds, start_mon, end_mon, 'time')
    ds = ds.load()
    ## run students t-test if anomaly == True
    if anomaly == True:    
        popmean = np.zeros([len(ds.level), len(ds.location)]) ## population mean
        ndays = len(ds.time) # number of unique days
        # calculate t-value based on ndays
        a_mean, tval_mask = ttest_1samp_new(a=ds, popmean=popmean, dim='time', n=ndays)

        ## write to netCDF
        out_fname = out_path + 'filtered_anomaly_composite_{0}_{1}_{2}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

        out_fname = out_path + 'filtered_anomaly_composite_tvals_{0}_{1}_{2}.nc'.format(varname, ssn, region)
        tval_mask.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    else:
        a_mean = ds.mean('time')
        out_fname = out_path + 'composite_{0}_{1}_{2}.nc'.format(varname, ssn, region)
        a_mean.to_netcdf(path=out_fname, mode = 'w', format='NETCDF4')

    return a_mean

In [9]:
%%time
## iterate through options
varname = 'sp_deg0l'
ssn_lst = ['DJF', 'MAM', 'JJA', 'SON', 'NDJFMA', 'MJJASO']

for i, ssn in enumerate(ssn_lst):
    print('Season:', ssn, 'Variable:', varname)
    tmp = compute_freezing_level_composites(varname, df['datetime'].values, ssn, region)

Season: DJF Variable: sp_deg0l
Season: MAM Variable: sp_deg0l
Season: JJA Variable: sp_deg0l
Season: SON Variable: sp_deg0l
Season: NDJFMA Variable: sp_deg0l
Season: MJJASO Variable: sp_deg0l
CPU times: user 25 s, sys: 2.05 s, total: 27.1 s
Wall time: 27.4 s


In [10]:
tmp

<xarray.Dataset>
Dimensions:    (location: 17, quantile: 3)
Coordinates:
    latitude   (location) float32 33.0 32.75 32.5 32.25 ... 29.5 29.25 29.0
    longitude  (location) float32 242.0 242.5 243.0 243.5 ... 249.0 249.5 250.0
  * quantile   (quantile) float64 0.1 0.5 0.9
Dimensions without coordinates: location
Data variables:
    sp         (quantile, location) float64 1.015e+05 1.015e+05 ... 9.443e+04
    deg0l      (quantile, location) float64 533.3 532.8 544.7 ... 634.1 649.7

In [16]:
%%time
## iterate through options
varname_lst = ['uvwq']
ssn_lst = ['DJF', 'MAM', 'JJA', 'SON', 'NDJFMA', 'MJJASO']
anom_lst = [True, False]

a = [varname_lst, ssn_lst, anom_lst]

option_lst = list(itertools.product(*a))
for i, lst in enumerate(option_lst):
    anomaly = lst[2]
    ssn = lst[1]
    varname = lst[0]
    print('Anomaly:', anomaly, 'Season:', ssn, 'Variable:', varname)
    tmp = compute_vertical_composites(varname, anomaly, ar_dates, ssn, region)

Anomaly: True Season: DJF Variable: uvwq
<xarray.Dataset>
Dimensions:    (location: 17, level: 23, time: 159)
Coordinates:
    longitude  (location) float32 -118.0 -117.5 -117.0 ... -111.0 -110.5 -110.0
    latitude   (location) float32 33.0 32.75 32.5 32.25 ... 29.5 29.25 29.0
  * level      (level) int32 1000 975 950 925 900 875 ... 350 300 250 225 200
  * time       (time) datetime64[ns] 2000-02-23 2000-02-24 ... 2023-01-17
    dayofyear  (time) int64 54 55 25 27 38 59 29 30 ... 361 364 365 1 15 16 17
Dimensions without coordinates: location
Data variables:
    q          (time, level, location) float64 0.008633 0.009625 ... 0.00282
    u          (time, level, location) float64 -0.06073 -0.8752 ... 17.97 16.71
    v          (time, level, location) float64 4.944 4.787 2.889 ... 23.12 22.5
    w          (time, level, location) float64 -0.006118 -0.1147 ... 0.1008
<xarray.Dataset>
Dimensions:    (location: 17, level: 23)
Coordinates:
    longitude  (location) float32 -118.0 -117.5 -